<a href="https://colab.research.google.com/github/Gokhman07/NLP/blob/master/M3_Part_I_Assignment_R_Gokhman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[link text](https://)##Assignment 3 Naïve Bayes and Sentiment Classification and Logistic Regression
Instructions
* Read the following Chapter 4: Naive Bayes and Sentiment Classification. Speech and Language Processing. Daniel Jurafsky & James H. Martin. Copyright © 2021. All rights reserved. Draft of September 21, 2021. I have tried to pull out relevant notes for you below, but it is encouraged that you read each chapter provided.
* Read the following Chapter 5: Logistic Regression. Speech and Language Processing. Daniel Jurafsky & James H. Martin. Copyright © 2021. All rights reserved. Draft of September 21, 2021. I have tried to pull out relevant notes for you below, but it is encouraged that you read each chapter provided.

Summary
Classification is one of the most important tasks of NLP and in machine learning. In NLP it often means the task of text categorization for both sentiment analysis, spam detection, and topic modeling. Naïve Bayes is often one of the first classification algorithms defined in NLP.  The intuition behind a classifier is lies at the underlying probability inferred by the Bayesian Inference, which uses Baye’s rule and conditional probabilities.

Here’s a reminder on Baye’s Rule:
P(y)=P(x)P(x)/(P(y))

We are saying “what is the probability of x given y”. Naïve Bayes is a generative model because there is an input that helps the model determine what the output could be. Said differently, “to train a generative model we first collect a large amount of data in some domain (e.g., think millions of images, sentences, or sounds, etc.) and then train a model to generate data like it.” [6]

So in the case of Naïve Bayes, we say given some word, what should be the class of the current word we are assessing? Contrastingly, discriminative models such as logistic regression, learn from features provided to the algorithm and then determine or predict what the class is. [7]


With Naïve Bayes, the assumption is that the probabilities are independent. We often call the Naïve Bayes classifier the bag-of-words approach. That’s because we are essentially throwing in the collection of words into a ‘bag’, selecting a word at random, and then calculating their frequency to use in the Bayesian Inference. Thus, context – the position of words -- is ignored and despite this, it turns out that the Naïve Bayes approach can be accurate and effective at determining whether an email is spam for example.

Back to bag of words. With bag of words, we assume that the position of the words are not relevant -- that dependency or context in the word phrase or sentence doesn’t matter. Relatedly, the naive Bayes assumption implies that the conditional probabilities are independent -- a rather strange assumption to make for words in a sentence! The equation for the naive Bayes classifier is outlined below:

You can use Naive Bayes by creating an index of words and walking through every word position in a test or corpus.


It can be useful to be able to classify new "test" documents using already classified "training" documents.  A common example is using a corpus of labeled spam and ham (non-spam) e-mails to predict whether or not a new document is spam.

For this Assignment, you can start with a spam/ham dataset, then predict the class of new documents (either withheld from the training dataset or from another source such as your own spam folder).   One example corpus:   https://spamassassin.apache.org/old/publiccorpus/

You may work alone or in a group on this project.  You're welcome to use any tools or approach that you like.  Due before our next meetup. Starter code provided below.

Test example is provided at the end.

Libraries you may wish to use

In [ ]:
import pandas as pd
import numpy as np
from os import makedirs, path, remove, rename, rmdir
from tarfile import open as open_tar
from shutil import rmtree
from urllib import request, parse
from glob import glob
from os import path
from re import sub
from email import message_from_file
from glob import glob
from sklearn.model_selection import StratifiedShuffleSplit
from collections import defaultdict
from functools import partial
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score)
from sklearn.model_selection import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
import gc
import nltk
import re

download corpus using the following functions

Note: you may need to mount your drive on google then run this location. See previous exercises.

In [ ]:
def download_corpus(dataset_dir: str = 'data'):
    base_url = 'https://spamassassin.apache.org'
    corpus_path = 'old/publiccorpus'
    files = {
        '20021010_easy_ham.tar.bz2': 'ham',
        '20021010_hard_ham.tar.bz2': 'ham',
        '20021010_spam.tar.bz2': 'spam',
        '20030228_easy_ham.tar.bz2': 'ham',
        '20030228_easy_ham_2.tar.bz2': 'ham',
        '20030228_hard_ham.tar.bz2': 'ham',
        '20030228_spam.tar.bz2': 'spam',
        '20030228_spam_2.tar.bz2': 'spam',
        '20050311_spam_2.tar.bz2': 'spam' }

    #creates the folders: downloads, ham and spam
    downloads_dir = path.join(dataset_dir, 'downloads')
    ham_dir = path.join(dataset_dir, 'ham')
    spam_dir = path.join(dataset_dir, 'spam')

    makedirs(downloads_dir, exist_ok=True)
    makedirs(ham_dir, exist_ok=True)
    makedirs(spam_dir, exist_ok=True)


    for file, spam_or_ham in files.items():
        # download files from URL of each specific .bz2 file
        url = parse.urljoin(base_url, f'{corpus_path}/{file}')
        tar_filename = path.join(downloads_dir, file)
        request.urlretrieve(url, tar_filename)

        #list e-mails in the compressed .bz2 file
        emails = []
        with open_tar(tar_filename) as tar:
            tar.extractall(path=downloads_dir)
            for tarinfo in tar:
                if len(tarinfo.name.split('/')) > 1:
                    emails.append(tarinfo.name)

        # move e-mails to ham or spam directory
        for email in emails:
            directory, filename = email.split('/')
            directory = path.join(downloads_dir, directory)

            if not path.exists(path.join(dataset_dir, spam_or_ham, filename)):
                rename(path.join(directory, filename),
                   path.join(dataset_dir, spam_or_ham, filename))

        rmtree(directory)

download_corpus()

#How many e-mails are classified in our dataset as either Spam or not Spam?


In [ ]:
#How many e-mails are classified in our dataset as either Spam or not Spam?
ham_dir = path.join('data', 'ham')
spam_dir = path.join('data', 'spam')

print('Number of Non-Spam E-mails:', len(glob(f'{ham_dir}/*')))
print('\nNumber of Spam E-mails:', len(glob(f'{spam_dir}/*')))

Number of Non-Spam E-mails: 6952

Number of Spam E-mails: 2399


Create DataFrame

In [ ]:
df=pd.DataFrame(columns=['mail','spam'])

In [ ]:
df

,mail,spam


In [ ]:
def add_mails(files, df, spam=False):
    if spam:
        status = 1
    else:
        status = 0

    for mail_file in files:
        with open(mail_file, 'r', encoding='latin-1') as file:
            text = file.read()
            df = df.append({'mail': text, 'spam': status}, ignore_index=True)

    return df


Add ham files

In [ ]:
df=add_mails(glob(f'{ham_dir}/*'), df, spam=False)

Add spam files

In [ ]:
df=add_mails(glob(f'{spam_dir}/*'), df, spam=True)

Suffle dataset

In [ ]:
df=df.sample(frac=1,ignore_index=True)

In [ ]:
df.head()

,mail,spam
0,Return-Path: <Online#3.19963.a7-m4d9lL5yDbfN5R...,0
1,From strange@nsk.yi.org Thu Sep 5 11:26:30 2...,0
2,From razor-users-admin@lists.sourceforge.net ...,0
3,From rpm-list-admin@freshrpms.net Mon Sep 2 ...,0
4,From rssfeeds@jmason.org Wed Oct 2 11:43:41 ...,0


In [ ]:
df.tail()

,mail,spam
9346,From dyporn@post.com Wed Sep 18 11:54:17 2002...,1
9347,From rpm-list-admin@freshrpms.net Tue Aug 27 ...,0
9348,From fork-admin@xent.com Wed Aug 28 17:25:50 ...,0
9349,From OWNER-NOLIST-SGODAILY*JM**NETNOTEINC*-COM...,1
9350,Received: from timhunt.net (ns1.timhunt.net [2...,1


In [ ]:
print(f'DataFrame shape is {df.shape}')

DataFrame shape is (9351, 2)


Cleaning function

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
 import email
 from nltk.corpus import stopwords
 from nltk.tokenize import word_tokenize
 import string
 from nltk.stem import PorterStemmer
 from bs4 import BeautifulSoup

In [ ]:
def extact_from_mail(text):
  # Parse the email
  msg = email.message_from_string(text)

  # Extract the content
  content = ""
  if msg.is_multipart():
    # If the email has multiple parts, iterate over them
    for part in msg.walk():
        # Check if the part is text/plain
        if part.get_content_type() == "text/plain":
            content += part.get_payload(decode=True).decode("latin-1")  # Decode the payload
  else:
    # If the email is not multipart, simply extract the text
    content = msg.get_payload(decode=True).decode("latin-1")  # Decode the payload

  return content


In [ ]:
def extact_from_html(text):

  soup=BeautifulSoup(text)
  content=soup.get_text()

  return content


In [ ]:
def cleaning_text(text):

  punctuations=list(string.punctuation)
  punctuations.append('...')

  #delete mail info
  text=extact_from_html(text)

    #removel email addres
  regex_remove_email =  r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b'

  text=re.sub(regex_remove_email,'',text, flags=re.MULTILINE)

  text=extact_from_mail(text)



  regex_remove_hyperlinks =r'((https|http)://|www\.)[A-Za-z0-9.:/-]*'

  text=re.sub(regex_remove_hyperlinks,'',text)
  #text=extact_from_html(text)
  text=text.lower()

   #remove web domains
  #text=re.sub(regex_remove_domain,'',text)
  #  remove hyperlinks, hashtags and 'b, and '>'


  # remove time_date in the begining

  regex_time_date = r'on\s+(.*?)\s+(.*?)\s+(.*?)\s+(wrote|said):'
  text=re.sub(regex_time_date,'',text)


  #remove time
  regex_remove_time = r'\d{2}:\d{2}(:\d{2})?(\s+(\+|-)\d{0,4})?'
  text=re.sub(regex_remove_time,'',text)




  #remove ip addresses
  regex_remove_ip=r'\d{1,4}.\d{1,4}.\d{1,4}.\d{1,4}'
  text=re.sub(regex_remove_ip,'',text)




  regex_hash = r'#'
  text=re.sub(regex_hash,'',text)

  regex_symbol = r'>'
  text=re.sub(regex_symbol,'',text)

  regex_string_beginning = r"^b['\"]"

  text=re.sub(regex_string_beginning,'',text)

  #tokens=word_tokenize(text)

  tokens=word_tokenize(text)

  non_symbol_pattern = r'^[!"#$%&/\'()*+,-.//:;<=>?@[\]^_`{|}~\\]+$'  # Matches if the token contains only non-word characters

  non_digit_pattern = r'^\d+((\.|,)\d*)?$'


  #the second step remove stopwords and punctuation
  stopwords_english=stopwords.words('english')

  cleaned_tokens=[]
  for token in tokens:
      if (token  in stopwords_english or token  in punctuations):
        continue
      elif bool((re.match(non_symbol_pattern,token))) or bool((re.match(non_digit_pattern,token))):
            continue
      else: cleaned_tokens.append(token)

  #stem tokens
  stemmer=PorterStemmer()
  for i,word in enumerate(cleaned_tokens):
    cleaned_tokens[i]=stemmer.stem(word)




  return cleaned_tokens

Apply function on the entire dataset

In [ ]:
df.loc[7749]['cleaned_mail']=cleaning_text(df.loc[7749]['mail'])

In [ ]:
df

,mail,spam,cleaned_mail
0,Return-Path: <Online#3.19963.a7-m4d9lL5yDbfN5R...,0,"[canning, spam, without, eat, real, mail, sear..."
1,From strange@nsk.yi.org Thu Sep 5 11:26:30 2...,0,"[make, wonder, rel, protect, smart, card, inte..."
2,From razor-users-admin@lists.sourceforge.net ...,0,"[today, 's, error, receiv, multipl, time, jul,..."
3,From rpm-list-admin@freshrpms.net Mon Sep 2 ...,0,"[want, thank, involv, make, rpm, avail, thank,..."
4,From rssfeeds@jmason.org Wed Oct 2 11:43:41 ...,0,"[url, date, suppli, joint, hear, senat, commit..."
...,...,...,...
9346,From dyporn@post.com Wed Sep 18 11:54:17 2002...,1,"[dyporn, post.com, wed, sep, 11:54:17, return-..."
9347,From rpm-list-admin@freshrpms.net Tue Aug 27 ...,0,"[rpm-list-admin, freshrpms.net, tue, aug, 10:3..."
9348,From fork-admin@xent.com Wed Aug 28 17:25:50 ...,0,"[fork-admin, xent.com, wed, aug, 17:25:50, ret..."
9349,From OWNER-NOLIST-SGODAILY*JM**NETNOTEINC*-COM...,1,"[owner-nolist-sgodaili, jm, netnoteinc, -com, ..."


In [ ]:
for i in range(len(df)):

  df.loc[i]['cleaned_mail']=cleaning_text( df.loc[i]['mail'])


In [ ]:
df.head()

,mail,spam,cleaned_mail
0,Return-Path: <Online#3.19963.a7-m4d9lL5yDbfN5R...,0,"[canning, spam, without, eat, real, mail, sear..."
1,From strange@nsk.yi.org Thu Sep 5 11:26:30 2...,0,"[make, wonder, rel, protect, smart, card, inte..."
2,From razor-users-admin@lists.sourceforge.net ...,0,"[today, 's, error, receiv, multipl, time, jul,..."
3,From rpm-list-admin@freshrpms.net Mon Sep 2 ...,0,"[want, thank, involv, make, rpm, avail, thank,..."
4,From rssfeeds@jmason.org Wed Oct 2 11:43:41 ...,0,"[url, date, suppli, joint, hear, senat, commit..."


In [ ]:
df.tail()

,mail,spam,cleaned_mail
9346,From dyporn@post.com Wed Sep 18 11:54:17 2002...,1,"[new, account, free, xxx, link, .org, non-prof..."
9347,From rpm-list-admin@freshrpms.net Tue Aug 27 ...,0,"[halloechen, dienstag, august, schrieben, sie,..."
9348,From fork-admin@xent.com Wed Aug 28 17:25:50 ...,0,"[gari, lawrenc, murphi, varieti, franchis, foo..."
9349,From OWNER-NOLIST-SGODAILY*JM**NETNOTEINC*-COM...,1,"[reliaquot, save, life, insur, life, chang, in..."
9350,Received: from timhunt.net (ns1.timhunt.net [2...,1,"[dear, friend, recent, survey, nielsen/netr, s..."


Applyy Naive Bayes

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [ ]:
list_preprocesses_mails=[]
for i in df['cleaned_mail']:
    list_preprocesses_mails.append(" ".join(i))

In [ ]:
list_preprocesses_mails.append(" ".join(test_tokens))

In [ ]:
bag_of_words=CountVectorizer().fit_transform(list_preprocesses_mails)

In [ ]:
bag_of_words

<9352x57040 sparse matrix of type '<class 'numpy.int64'>'
	with 924529 stored elements in Compressed Sparse Row format>

In [ ]:
print(f'Shape of bag of words: {bag_of_words.shape}')

Shape of bag of words: (9352, 57040)


Data split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(bag_of_words[:-1], df['spam'].astype('int'), test_size=0.20, random_state=0)

Model fit

In [ ]:
mnb=MultinomialNB()
mnb.fit(X_train,y_train)

MultinomialNB()

Predict test values

In [ ]:
y_pred=mnb.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test,y_pred))
print("Confusion matrix:")
print(metrics.confusion_matrix(y_test,y_pred))
print("Accuracy:")
print(metrics.accuracy_score(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1389
           1       0.98      0.91      0.95       482

    accuracy                           0.97      1871
   macro avg       0.98      0.95      0.96      1871
weighted avg       0.97      0.97      0.97      1871

Confusion matrix:
[[1382    7]
 [  42  440]]
Accuracy:
0.9738107963655799


The following email is a test email. You can take this and test your classifier to see if it predicts spam or not.



In [ ]:
spam_email = """
Subject: Get Rich Quick!

Dear Friend,

Congratulations! You've been selected to participate in an exclusive opportunity to make thousands of dollars from the comfort of your own home. Our revolutionary system guarantees quick and easy cash with minimal effort.

No more struggling to pay bills or worrying about financial security. With our proven method, you can start earning massive amounts of money in no time.

Here's what some of our satisfied customers have to say:
- "I was skeptical at first, but I'm now living my dream life thanks to this incredible system!" - John S.
- "I never thought making money online could be this simple. It's changed my life!" - Sarah L.

Don't miss out on this limited-time offer. Act now to secure your spot and start enjoying a life of financial freedom.

Click the link below to get started:
www.getrichquick.com

Remember, this opportunity is exclusive and won't last long. Take control of your financial future today!

Best regards,
The Get Rich Quick Team
"""


In [ ]:
test_tokens=cleaning_text(spam_email)

In [ ]:
mnb.predict(bag_of_words[-1:][0])[0]

1

As we classifier on text example showed that spam mail is spam